In [ ]:
# Code by https://www.reddit.com/user/JWson/

In [1]:
import cv2
import os
 
import numpy as np
 
class ImageRecorder:
    def __init__(self, x_min, y_min, x_max, y_max, w, h):
        self.x_min = x_min
        self.x_max = x_max
        self.y_min = y_min
        self.y_max = y_max
       
        self.w = w
        self.h = h
       
        self.arr = np.zeros((h, w), dtype=np.uint64)
   
    def record(self, x, y):
        j_nominal = int((x - self.x_min)*self.w/(self.x_max - self.x_min))
        j = max( 0, min( self.w-1, j_nominal ) )
       
        y_prime = self.y_min + self.y_max - y
        i_nominal = int((y_prime - self.y_min)*self.h/(self.y_max-self.y_min) )
        i = max( 0, min( self.h-1, i_nominal ) )
       
        self.arr[i,j] += 1
   
    def record_stack(self, x, y):
        factor_x = self.w/(self.x_max - self.x_min)
        V = (x - self.x_min)*factor_x
        U = self.h*( 1.0 - (y - self.y_min)/(self.y_max - self.y_min) )
       
        np.add.at(self.arr, ( U.astype(int),V.astype(int) ), 1)
   
    def save_image(self, out_folder, fname):
        px = np.copy(self.arr)
        px = np.sqrt(px)
        px = px/np.max(px)
        px = px*px
        px = px*255
       
        cv2.imwrite(os.path.join(out_folder, fname+".png"), px)

ModuleNotFoundError: No module named 'cv2'

In [2]:
import cv2
import os
 
image_folder = 'frames'
video_name = 'my-video.mp4'
 
images = [img for img in os.listdir(image_folder) if img.endswith(".png")]
images = sorted(images)
frame = cv2.imread(os.path.join(image_folder, images[0]))
height, width, layers = frame.shape
 
video = cv2.VideoWriter(os.path.join("vid", video_name), 0, 60, (width,height))
 
for image in images:
    video.write(cv2.imread(os.path.join(image_folder, image)))
 
cv2.destroyAllWindows()
video.release()

ModuleNotFoundError: No module named 'cv2'

In [3]:
import os
import numpy as np
 
from ImageRecorder import ImageRecorder
from scipy.spatial.transform import Rotation as rot
from time import process_time as pt
from math import pi, acos, sin, cos
 
 
out_folder = "frames"
 
try:
    os.mkdir(out_folder)
except FileExistsError:
    print ("Directory", out_folder, "already exists, skipping creation.")
 
octo_nodes = np.array([[ 0, 0, 1],
                       [ 0, 0,-1],
                       [ 0, 1, 0],
                       [ 0,-1, 0],
                       [ 1, 0, 0],
                       [-1, 0, 0]])
 
def rotated_nodes(nodes, angle, axis):
    """Rotate nodes matrix by angle along given axis"""
    rotation = rot.from_euler(axis, angle, degrees=False)
    M = rotation.as_dcm()
    new_nodes = np.matmul(M, np.transpose(nodes))
    return np.transpose(new_nodes)
 
def rotate_nodes_rotvec(nodes, rotvec):
    rotation = rot.from_rotvec(rotvec)
    M = rotation.as_dcm()
    new_nodes = np.matmul(M, np.transpose(nodes))
    return np.transpose(new_nodes)
 
def chaos_iterate(x, y, nodes):
    n_points = x.shape[0]
    n_nodes = nodes.shape[0]
   
    target_indices = np.random.randint(0, n_nodes, size=n_points, dtype=int)
    targets = nodes[target_indices]
    u, v = targets[:,0], targets[:,1]
   
    return (x + u)*0.5, (y + v)*0.5
 
def record_chaos(recorder, nodes, n_iters=1000, n_points=10000, windup=10):
    x = np.random.rand(n_points)
    y = np.random.rand(n_points)
   
    for k in range(n_iters + windup):
        x, y = chaos_iterate(x, y, nodes)
       
        if k+1 > windup:
            recorder.record_stack(x,y)
   
    return recorder
 
def pad_k(k, length=7):
    s = str(k)
    while len(s) < length:
        s = "0" + s
    return s
   
 
 
octo_nodes_flop = rotated_nodes(octo_nodes, pi/4.0, "z")
 
total_time = 90.0 # seconds
 
omega_1 = 1.5 # deg/s
omega_2 = 9.0 # deg/s
 
frame_rate = 60.0 # fps
dt = 1.0/frame_rate # sec
n_frames = int(total_time/dt)
 
d_theta = dt*omega_2*pi/180.0
 
current_nodes = np.copy(octo_nodes_flop)
 
t0=pt()
 
for k in range(n_frames):
    print(k, "/", n_frames)
   
    current_time = k*dt
    theta_rotvec = k*dt*omega_1*pi/180.0
    current_rotvec = np.array([sin(theta_rotvec), cos(theta_rotvec), 0.0])
   
    current_nodes = rotate_nodes_rotvec(current_nodes, d_theta*current_rotvec)
   
    current_nodes_2d = current_nodes[:,:2]
   
    current_recorder = ImageRecorder(-1.1, -1.1, 1.1, 1.1, 1000, 1000)
    current_recorder = record_chaos(current_recorder, current_nodes)
    current_recorder.save_image(out_folder, pad_k(k))
 
t1=pt()
print("Time taken is", t1-t0, "seconds")

ModuleNotFoundError: No module named 'ImageRecorder'